# Vivino web scraping with Pycurl
> Vivino has a database of 3 mln wines... this notebook won't allow you to scrape the entire website and it is mainly used to understand how to get JSON file with Pycurl and build a pandas dataframe from complex JSON. This code works on a sample of 2500 wines loaded from 100 vivino pages.

## Code and Dataset to get most of the wines
> please refer to this code if you want to see how i have managed memory usage and saved multiple dataframes:  
Link to the final dataset:

## Premises
> When scraping data, the most important part is the website structure analysis, in fact i have spent many hours navigating vivino website to get as much info as possible. I would like to guide you through some of the problem i have faced.

## Javascript generated content
> For complex websites you cannot directly scrape related HTML because most of the time there will be a JavaScript script generating informations.
I have started considering Selenium to simulate a webdriver which would have fully loaded HTML info and then parse all this info with beautiful soup.

## Selenium
>The first problem of vivino website using Selenium was to fully load the wine list scrolling to the bottom, this require a scroll call with selenium until the end is reached, however by using this strategy you have to perform multiple calls and there may be the risk of being banned.

## Find the most important JSON (there must be one!)
>When navigating the website i was not able to find Json file with most of the info, this is because you get such file only after you start scrolling to the end of the first page.  
"Wines info are here, there must be a file..." after a while i have found the most beautiful Json file related to wines. 725 lines of Json for each single wine.  
I immediately find out that the harderst part was not to scrape the website but to become a wine expert to take only most useful info from Json.

# Tables that will be created from this code
- vintage (this is the main table)
- region_most_used_grapes
- flavor
   - flavor_primary_keywords
   - flavor_secondary_keywords
- prices
- food
- grapes

# <b> <span style='color:#0EB1D2'>Step 1</span> <span style='color:#0EB1D2'>|</span> <span style='color:#ED6A5E ;font-style: italic;animation: pulse 2s infinite;'>Import Libraries</span></b>

In [1]:
import time
import pycurl
import certifi
from io import BytesIO
import json
import pandas as pd

In [2]:
#I just like to see all columns and more rows with full content, this option will be reset when required
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', None)

# <b> <span style='color:#0EB1D2'>Step 2</span> <span style='color:#0EB1D2'>|</span> <span style='color:#ED6A5E ;font-style: italic;animation: pulse 2s infinite;'>Scrapig with Pycurl</span></b>

In [3]:
%%time
n = 0
pages_available = True
wines_collection = []

for i in range(100):
# Creating a buffer as the cURL is not allocating a buffer for the network response
    buffer = BytesIO()

# Initialising the curl object
    curl = pycurl.Curl()

# Setting the request URL, the user agent, and the buffer for the response
    url = f'https://www.vivino.com/api/explore/explore?country_code=IT&currency_code=EUR&grape_filter=varietal&min_rating=1&order_by=price&order=desc&price_range_max=500+&price_range_min=0&wine_type_ids%5B%5D=1&wine_type_ids%5B%5D=2&wine_type_ids%5B%5D=3&wine_type_ids%5B%5D=24&wine_type_ids%5B%5D=7&wine_type_ids%5B%5D=4&page={n}&language=en'
    curl.setopt(pycurl.URL, url)
    curl.setopt(pycurl.USERAGENT, "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0")
    curl.setopt(pycurl.WRITEDATA, buffer)
    curl.setopt(pycurl.CAINFO, certifi.where())

# Performing the actual request
    curl.perform()

# Accessing the response code and adding 1 to n only if successful, this step is not necessary it allows you to check each step
    if curl.getinfo(pycurl.RESPONSE_CODE) == 200:
        #print(f'Success : {n} {curl.getinfo(pycurl.RESPONSE_CODE)}', )
        n +=1
    else:
        print(f'Not Success{curl.getinfo(pycurl.RESPONSE_CODE)}')

# Ending the session and freeing the resources
    curl.close()
    
#Convert Buffer from Byte to string to Dict
    page_dict = json.loads(buffer.getvalue().decode())

#page_dict is now a dictionaire containing three main dictionaries, the one we want is "explore_vintage", it contains another dict "matches"
#matches is a list of dict
    for i in page_dict['explore_vintage']['matches']:
        wines_collection.append(i)
    
    if n%10 == 0:
        print(len(wines_collection))


250
500
750
1000
1250
1500
1750
2000
2250
2500
CPU times: user 5.61 s, sys: 1.01 s, total: 6.62 s
Wall time: 1min 35s


# <b> <span style='color:#0EB1D2'>Step 3</span> <span style='color:#0EB1D2'>|</span> <span style='color:#ED6A5E ;font-style: italic;animation: pulse 2s infinite;'>Optional - Check saved content in Buffer</span></b>
>When we save our scraped content into a buffer this will be in bytes format, we use decode to convert it into string and then json library to finally get a dictionary we can work with

In [ ]:
#buffer getvalue() now store informations we need in a bytes format
print(type(buffer.getvalue()))

#hence we will decode bytes into string
string = buffer.getvalue().decode()
print(type(string))

#at this stage we can  convert our string into a dictionary using json library
wines_info = json.loads(string)
print(type(wines_info))



In [ ]:
#We have three main keys
print(wines_info.keys())
#explore vintage has more keys
print(wines_info['explore_vintage'].keys())
#matches provides a list of values, let's see keys of the first value of this list
print(wines_info['explore_vintage']['matches'][0].keys())
print(wines_info['explore_vintage']['matches'][0]['vintage'].keys())

In [ ]:
#let's take the first wine of the list JSON, you can copy and paste it in a Json formatter or just use visual studio
wines_collection[0]

# <b> <span style='color:#0EB1D2'>Step 3</span> <span style='color:#0EB1D2'>|</span> <span style='color:#ED6A5E ;font-style: italic;animation: pulse 2s infinite;'>Pandas Dataframe</span></b>

## Main Table
as you can see we have now a very clear dataset with lots of variables, please note that the following variables are still a list of dictionaries:
- prices
- most_used_grapes
- flavor
- food
- grapes

In [6]:
pd.reset_option('display.max_colwidth')
vintage = pd.json_normalize(wines_collection)
vintage.head()

,prices,vintage.id,vintage.seo_name,vintage.name,vintage.statistics.status,vintage.statistics.ratings_count,vintage.statistics.ratings_average,vintage.statistics.labels_count,vintage.statistics.wine_ratings_count,vintage.statistics.wine_ratings_average,vintage.statistics.wine_status,vintage.image.location,vintage.image.variations.bottle_large,vintage.image.variations.bottle_medium,vintage.image.variations.bottle_medium_square,vintage.image.variations.bottle_small,vintage.image.variations.bottle_small_square,vintage.image.variations.label,vintage.image.variations.label_large,vintage.image.variations.label_medium,vintage.image.variations.label_medium_square,vintage.image.variations.label_small_square,vintage.image.variations.large,vintage.image.variations.medium,vintage.image.variations.medium_square,vintage.image.variations.small_square,vintage.wine.id,vintage.wine.name,vintage.wine.seo_name,vintage.wine.type_id,vintage.wine.vintage_type,vintage.wine.is_natural,vintage.wine.region.id,vintage.wine.region.name,vintage.wine.region.name_en,vintage.wine.region.seo_name,vintage.wine.region.country.code,vintage.wine.region.country.name,vintage.wine.region.country.native_name,vintage.wine.region.country.seo_name,vintage.wine.region.country.currency.code,vintage.wine.region.country.currency.name,vintage.wine.region.country.currency.prefix,vintage.wine.region.country.currency.suffix,vintage.wine.region.country.regions_count,vintage.wine.region.country.users_count,vintage.wine.region.country.wines_count,vintage.wine.region.country.wineries_count,vintage.wine.region.country.most_used_grapes,vintage.wine.region.background_image,vintage.wine.winery.id,vintage.wine.winery.name,vintage.wine.winery.seo_name,vintage.wine.winery.status,vintage.wine.winery.background_image,vintage.wine.taste.structure.acidity,vintage.wine.taste.structure.fizziness,vintage.wine.taste.structure.intensity,vintage.wine.taste.structure.sweetness,vintage.wine.taste.structure.tannin,vintage.wine.taste.structure.user_structure_count,vintage.wine.taste.structure.calculated_structure_count,vintage.wine.taste.flavor,vintage.wine.statistics,vintage.wine.style.id,vintage.wine.style.seo_name,vintage.wine.style.regional_name,vintage.wine.style.varietal_name,vintage.wine.style.name,vintage.wine.style.background_image,vintage.wine.style.image,vintage.wine.style.description,vintage.wine.style.blurb,vintage.wine.style.interesting_facts,vintage.wine.style.body,vintage.wine.style.body_description,vintage.wine.style.acidity,vintage.wine.style.acidity_description,vintage.wine.style.country.code,vintage.wine.style.country.name,vintage.wine.style.country.native_name,vintage.wine.style.country.seo_name,vintage.wine.style.country.currency.code,vintage.wine.style.country.currency.name,vintage.wine.style.country.currency.prefix,vintage.wine.style.country.currency.suffix,vintage.wine.style.country.regions_count,vintage.wine.style.country.users_count,vintage.wine.style.country.wines_count,vintage.wine.style.country.wineries_count,vintage.wine.style.country.most_used_grapes,vintage.wine.style.wine_type_id,vintage.wine.style.food,vintage.wine.style.grapes,vintage.wine.style.region.id,vintage.wine.style.region.name,vintage.wine.style.region.name_en,vintage.wine.style.region.seo_name,vintage.wine.style.region.country.code,vintage.wine.style.region.country.name,vintage.wine.style.region.country.native_name,vintage.wine.style.region.country.seo_name,vintage.wine.style.region.country.currency.code,vintage.wine.style.region.country.currency.name,vintage.wine.style.region.country.currency.prefix,vintage.wine.style.region.country.currency.suffix,vintage.wine.style.region.country.regions_count,vintage.wine.style.region.country.users_count,vintage.wine.style.region.country.wines_count,vintage.wine.style.region.country.wineries_count,vintage.wine.style.region.country.most_used_grapes,vintage.wine.style.region.parent_id,vintage.wine.style.region.background_image.location,vintage.wine.style.region.background_

## Subtables with json_normalize
transform list of dictionaries within the main dataset to many subdatasets that can be used by joining with the wine id (vintage.id)

### vintage.wine.region.country.most_used_grapes

In [7]:
pd.reset_option('display.max_colwidth')
region_most_used_grapes = pd.json_normalize(wines_collection,
                                     record_path = ['vintage', 'wine', 'region', 'country', 'most_used_grapes'],
                                     record_prefix = 'vintage_wine_region_country_MostUsedGrapes_', meta = [['vintage', 'id']])
region_most_used_grapes.head(10)

,vintage_wine_region_country_MostUsedGrapes_id,vintage_wine_region_country_MostUsedGrapes_name,vintage_wine_region_country_MostUsedGrapes_seo_name,vintage_wine_region_country_MostUsedGrapes_has_detailed_info,vintage_wine_region_country_MostUsedGrapes_wines_count,vintage_wine_region_country_MostUsedGrapes_parent_grape_id,vintage.id
0,14,Pinot Noir,pinot-noir,True,572334,NaN,156099154
1,5,Chardonnay,chardonnay,True,604208,NaN,156099154
2,10,Merlot,merlot,True,566719,NaN,156099154
3,15,Riesling,riesling,True,262136,NaN,156856757
4,299,Spätburgunder,spatburgunder,False,50164,14.0,156856757
5,355,Weissburgunder,weissburgunder,False,33464,36.0,156856757
6,14,Pinot Noir,pinot-noir,True,572334,NaN,159343591
7,5,Chardonnay,chardonnay,True,604208,NaN,159343591
8,10,Merlot,merlot,True,566719,NaN,159343591
9,14,Pinot Noir,pinot-noir,True,572334,NaN,1746707


### vintage.wine.taste.flavor
flavor has two other nested dictionaries primary_key and secondary_key to get the specific main flavor components, we can associate them as flavor notes, in my opinion the flavor group would be enough

In [8]:
vintage['vintage.wine.taste.flavor'].isna().sum()

np.int64(0)

In [9]:
flavor = pd.json_normalize(wines_collection,
                                     record_path = ['vintage', 'wine', 'taste', 'flavor'], meta = [['vintage', 'id']])
flavor.head(10)

,group,primary_keywords,secondary_keywords,stats.count,stats.score,vintage.id
0,red_fruit,"[{'id': 348, 'name': 'raspberry', 'count': 34}...","[{'id': 38, 'name': 'black cherry', 'count': 1...",87,11822,156099154
1,earth,"[{'id': 242, 'name': 'leather', 'count': 33}, ...","[{'id': 422, 'name': 'tobacco', 'count': 3}, {...",73,8582,156099154
2,oak,"[{'id': 292, 'name': 'oak', 'count': 11}, {'id...","[{'id': 242, 'name': 'leather', 'count': 33}, ...",69,4570,156099154
3,black_fruit,"[{'id': 38, 'name': 'black cherry', 'count': 1...","[{'id': 341, 'name': 'prune', 'count': 2}]",49,6034,156099154
4,non_oak,"[{'id': 62, 'name': 'brioche', 'count': 1}, {'...","[{'id': 384, 'name': 'smoke', 'count': 7}, {'i...",30,603,156099154
5,spices,"[{'id': 320, 'name': 'pepper', 'count': 6}, {'...","[{'id': 434, 'name': 'vanilla', 'count': 2}, {...",21,2195,156099154
6,floral,"[{'id': 322, 'name': 'perfume', 'count': 5}, {...","[{'id': 381, 'name': 'savory', 'count': 2}]",19,2050,156099154
7,microbio,"[{'id': 238, 'name': 'lager', 'count': 1}, {'i...","[{'id': 284, 'name': 'mushroom', 'count': 9}, ...",12,700,156099154
8,citrus_fruit,"[{'id': 298, 'name': 'orange', 'count': 3}, {'...",NaN,7,700,156099154
9,dried_fruit,"[{'id': 341, 'name': 'prune', 'count': 2}, {'i...","[{'id': 229, 'name': 'jam', 'count': 1}]",5,394,156099154


### Prices
wine price is already included into the main table as a float value, this table contains prices related with the id of the vendor

In [10]:
vintage['prices'].isna().sum()

np.int64(0)

In [11]:
pd.reset_option('display.max_colwidth')
prices = pd.json_normalize(wines_collection,
                                     record_path = 'prices',
                                     record_prefix = 'prices_', meta = [['vintage', 'id']])
prices.head(10)

,prices_id,prices_merchant_id,prices_amount,prices_discounted_from,prices_discount_percent,prices_type,prices_sku,prices_url,prices_visibility,prices_bottle_type_id,prices_xms,prices_xmphs,prices_price_group_id,prices_vfm_score,prices_currency.code,prices_currency.name,prices_currency.prefix,prices_currency.suffix,prices_vfm_category.id,prices_vfm_category.title,prices_vfm_category.subtitle,prices_vfm_category.barometer_position,prices_bottle_type.id,prices_bottle_type.name,prices_bottle_type.short_name,prices_bottle_type.short_name_plural,prices_bottle_type.volume_ml,vintage.id
0,35134561,18999,50960.00,NaN,NaN,vc,MTH2204156,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,156099154
1,34993404,18999,16621.89,NaN,NaN,vc,CTH493,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,156856757
2,33381495,18999,13613.25,NaN,NaN,vc,NND2559,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,159343591
3,22633878,18999,13106.10,NaN,NaN,vc,NND2117,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,1746707
4,22633879,18999,11989.95,NaN,NaN,vc,NND2555,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,14162009
5,36150812,18999,11892.30,NaN,NaN,vc,NND2116,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,1759463
6,26301254,18999,11239.20,NaN,NaN,vc,NND2558,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,156099048
7,36150813,18999,10943.10,NaN,NaN,vc,NND2118,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,162902104
8,29295725,18999,10943.10,NaN,NaN,vc,NND2114,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,2893744
9,22633877,18999,10943.10,NaN,NaN,vc,NND2115,https://www.soloprodottiitaliani.it/spec_colle...,1,1,0.541159,False,[xom],0.0,EUR,Euros,€,None,0,Value Unavailable,We’re still gathering data to determine this w...,None,1,Bottle (0.75l),bottle,bottles,750,1921204


## Subtables without json_normalize

#### Flavor primary_keyboards and secondary_keywords
some variables like, flavor primary_keyboards and secondary_keywords are not always populated and it is not possible to use json_normalize.

we can still create a dataframe by looping into flavor primary_keywords but i believe it is better to keep the list into the flavor dataframe and access it if needed. moreover to build ML model you don't really need the specific level of each flavor, you can just use the flavor category and use the relative stats.count as flavor weight for that wine.

In [12]:
def dict_to_table(data, column, key):
    empty_list = []
    for i,j in zip(data[column], data[key]):
        try:
            for n in i:
                n[key] = j
                empty_list.append(n)
        except:
            pass
    return empty_list

In [13]:
#check null values, if null > 0 we cannot use json_normalize
flavor['primary_keywords'].isna().sum()

np.int64(1100)

In [14]:
flavor_primary_keywords = pd.DataFrame(dict_to_table(data = flavor, column = 'primary_keywords', key = 'vintage.id' ))
flavor_primary_keywords.head(100)

,id,name,count,vintage.id
0,348,raspberry,34,156099154
1,93,cherry,28,156099154
2,400,strawberry,25,156099154
3,354,red fruit,18,156099154
4,352,red cherry,5,156099154
...,...,...,...,...
95,298,orange,1,156856757
96,329,pineapple,2,156856757
97,123,cream,1,156856757
98,35,biscuit,1,156856757


In [15]:
#check null values, if null > 0 we cannot use json_normalize
flavor['secondary_keywords'].isna().sum()

np.int64(6912)

In [16]:
flavor_secondary_keywords = pd.DataFrame(dict_to_table(data = flavor, column = 'secondary_keywords', key = 'vintage.id' ))
flavor_secondary_keywords.head(100)

,id,name,count,vintage.id
0,38,black cherry,14,156099154
1,335,plum sauce,2,156099154
2,229,jam,1,156099154
3,422,tobacco,3,156099154
4,134,dark chocolate,3,156099154
...,...,...,...,...
95,113,cocoa,15,1746707
96,88,charcoal,2,1746707
97,14,ash,2,1746707
98,65,brown sugar,2,1746707


### Food

In [17]:
vintage['vintage.wine.style.food'].isna().sum()

np.int64(43)

In [18]:
food = pd.DataFrame(dict_to_table(data = vintage, column = 'vintage.wine.style.food', key = 'vintage.id' ))
food.head(100)

,id,name,weight,background_image,seo_name,vintage.id
0,4,Beef,0.5,{'location': '//images.vivino.com/backgrounds/...,beef,156099154
1,9,Veal,0.5,{'location': '//images.vivino.com/backgrounds/...,veal,156099154
2,11,"Game (deer, venison)",0.0,{'location': '//images.vivino.com/backgrounds/...,game,156099154
3,20,Poultry,0.5,{'location': '//images.vivino.com/backgrounds/...,poultry,156099154
4,4,Beef,0.5,{'location': '//images.vivino.com/backgrounds/...,beef,159343591
...,...,...,...,...,...,...
95,11,"Game (deer, venison)",0.0,{'location': '//images.vivino.com/backgrounds/...,game,87613428
96,20,Poultry,0.5,{'location': '//images.vivino.com/backgrounds/...,poultry,87613428
97,4,Beef,0.5,{'location': '//images.vivino.com/backgrounds/...,beef,156099154
98,9,Veal,0.5,{'location': '//images.vivino.com/backgrounds/...,veal,156099154


### Grapes

In [19]:
vintage['vintage.wine.style.grapes'].isna().sum()

np.int64(62)

In [20]:
grapes = pd.DataFrame(dict_to_table(data = vintage, column = 'vintage.wine.style.grapes', key = 'vintage.id' ))
grapes.head(100)

,id,name,seo_name,has_detailed_info,wines_count,parent_grape_id,vintage.id
0,14,Pinot Noir,pinot-noir,True,572334,NaN,156099154
1,2,Cabernet Sauvignon,cabernet-sauvignon,True,801751,NaN,159343591
2,3,Cabernet Franc,cabernet-franc,True,269578,NaN,159343591
3,9,Malbec,malbec,True,219735,NaN,159343591
4,10,Merlot,merlot,True,566719,NaN,159343591
...,...,...,...,...,...,...,...
95,51,Carménère,carmenere,True,50087,NaN,14162009
96,2,Cabernet Sauvignon,cabernet-sauvignon,True,801751,NaN,1759463
97,3,Cabernet Franc,cabernet-franc,True,269578,NaN,1759463
98,9,Malbec,malbec,True,219735,NaN,1759463
